In [2]:
## prereqs
import pandas as pd

# Data Dig, Ask me Questions / for data etc...

**srimmele@gmail.com**

## FCC Form 477 Data

Download and unzip from here: https://www.fcc.gov/form477/BroadbandData/Fixed/Dec15/Version%201/NY-Fixed-Dec2015.zip

**FCC Data Codes and Descriptions can be mystifying:** Check out their [resources](https://www.fcc.gov/general/form-477-resources-filers) for more info.

Quick-link to [FCC Data Dictionary](https://www.fcc.gov/general/explanation-broadband-deployment-data)

In [3]:
FCC = pd.read_csv('../inputs/NY-Fixed-Dec2015-v1.csv')
        

IOError: File ../inputs/NY-Fixed-Dec2015-v1.csv does not exist

## Census Block Codes

The FCC organizes broadband access data by Census Bureau groupings, like so:

![Census ID Codes](http://proximityone.com/cv_dr_graphics/geo_block.jpg)

The counties of New York City are:

  *    005 - Bronx
  *   047 - Kings (Brooklyn)
  *  061 - New York (Manhattan)
  * 081 - Queens
  * 085 - Richmond (Staten Island)



[More detail on Census Blocks here](http://proximityone.com/geo_blocks.htm)

In [ ]:

block_code = FCC.BlockCode.astype(str)

FCC['STATE_CODE'] = block_code.apply(lambda x: x[0:2])
FCC['COUNTY_CODE'] = block_code.apply(lambda x: x[2:5].lstrip("0"))
FCC['TRACT'] = block_code.apply(lambda x: x[5:11].lstrip("0"))
FCC['BLOCK'] = block_code.apply(lambda x: x[11:])
FCC['BLOCK_GROUP'] = block_code.apply(lambda x: x[11])
FCC['GEO_ID'] = block_code.apply(lambda x: x[:12])  # Unique to block group, like the GeoJSON used for mapping



FCC.head(1).T

## NYC Neighborhoods

[NYC DCP](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page) provides a handy mapping of 2010 Census Tracts to NYC Neighborhood Tabulation Areas (e.g., 'Allerton-Pelham Gardens')



In [ ]:
NTA_to_CT = pd.read_excel('https://www1.nyc.gov/assets/planning/download/office/data-maps/nyc-population/census2010/nyc2010census_tabulation_equiv.xlsx', \
                          sheetname='NTA to 2010 CT equivalency',header=3 ,  \
                          converters = {'2010 NYC Borough Code': str, '2010 Census Bureau FIPS County Code':str ,\
                                      '2010 Census Tract':str})

## Have to fix some of the default formatting and drop an empty row
NTA_to_CT.rename(columns= {'Unnamed: 6':'NTA_Name'}, inplace=True)
NTA_to_CT.drop([0], inplace=True)

NTA_to_CT['BC'] = NTA_to_CT['2010 NYC Borough Code'].apply(lambda x: x.lstrip('0'))  # drop leading zeros
NTA_to_CT['CC'] = NTA_to_CT['2010 Census Bureau FIPS County Code'].apply(lambda x: x.lstrip('0'))  # drop leading zeros
NTA_to_CT['CT'] = NTA_to_CT['2010 Census Tract'].apply(lambda x: x.lstrip('0'))  # drop leading zeros

NTA_to_CT.head()

In [ ]:
FCC.head()

## Neighborhood level dataset

*FCC_Merged* is a dataset of provider-reported internet access data from FCC Form 477, at the Census Block Level. Each record includes other groupings (Census Block Group, Census Tract, NYC Neighborhood Tabulation Area, Borough) as well.

You can filter for different categories e.g, "Consumer Internet" and "Fiber" as you see fit, an example is shown below.

**FCC Data Codes and Descriptions can be mystifying:** Check out their [resources](https://www.fcc.gov/general/form-477-resources-filers) for more info 

In [ ]:
FCC_merged = FCC.merge(NTA_to_CT, left_on=['COUNTY_CODE','TRACT'],right_on=['CC','CT'],how='inner')

print str(len(FCC_merged.BlockCode.unique())) + ' Blocks in base dataset (citywide)'

'''
Consumer: (0/1) where 1 = Provider can or does offer consumer/mass market/residential service in the block
'''
FCC_merged = FCC_merged[FCC_merged.Consumer == 1]

print str(len(FCC_merged.BlockCode.unique())) + ' blocks with consumer internet'

'''
TechCode: 10 is XDSL, 50 is Fiber to End User
'''
FCC_merged = FCC_merged[FCC_merged.TechCode == 50]

print str(len(FCC_merged.BlockCode.unique())) + ' blocks with consumer fiber internet'

FCC_merged.head(1).T


## Example Use

Here's a borough comparison of Maximum Advertised Download Speeds

In [ ]:
%matplotlib inline 
import seaborn

# to show charts in the notebook

FCC_merged.groupby(['Borough']).mean()['MaxAdDown'].sort_values().plot(kind='bar')